In [38]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

In [40]:
%matplotlib inline

In [41]:
TEXT_COLOUR = {
    'PURPLE':'\033[95m',
    'CYAN':'\033[96m',
    'DARKCYAN':'\033[36m',
    'BLUE':'\033[94m',
    'GREEN':'\033[92m',
    'YELLOW':'\033[93m',
    'RED':'\033[91m',
    'BOLD':'\033[1m',
    'UNDERLINE':'\033[4m',
    'END':'\033[0m'
}

def print_bold(*msgs):
    print(TEXT_COLOUR['BOLD'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def print_green(*msgs):
    print(TEXT_COLOUR['GREEN'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def print_error(*msgs):
    print(TEXT_COLOUR['RED'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def wrap_green(msg):
    return TEXT_COLOUR['GREEN'] + msg + TEXT_COLOUR['END']

def wrap_red(msg):
    return TEXT_COLOUR['RED'] + msg + TEXT_COLOUR['END']

def up_down_str(val):
    msg = str(val)
    if val > 0:
        msg = wrap_green(msg)
    elif val < 0:
        msg = wrap_red(msg)
    return msg

In [42]:
exp='bert-base'
num_layers = 12

In [43]:
tasks = ["CoLA","SST-2","MRPC","STS-B","QQP","MNLI", "MNLI-MM", "QNLI", "RTE"]

metrics = {
    "CoLA":["mcc"],
    "MNLI":["acc"],
    "MNLI-MM":["acc"],
    "MRPC":["f1"],
    "QNLI":["acc"],
    "QQP":["f1"],
    "RTE":["acc"],
    "SST-2":["acc"],
    "STS-B":["spearmanr"],
    "WNLI":["acc"] #temp
}

reported_in_paper = {
    "CoLA":0.00,
    "MNLI":0.00,
    "MNLI-MM":0.0,
    "MRPC":0.00,
    "QNLI":0.00,
    "QQP":0.00,
    "RTE":0.00,
    "SST-2":0.00,
    "STS-B":0.00,
    "WNLI":0.00
}

In [44]:

def get_average_val(lines):
    reported = []
    for line in lines:
        val = float(line.split()[1])
        if val != 0:
            reported.append(val)
    out = 0
    if len(reported) != 0:
        reported.sort(reverse = True)
        candidates = [reported[0]]
        for j in range(1, len(reported)):
            if reported[j] > 0.9 * reported[0]:
                candidates.append(reported[j])
        out = np.mean(candidates)
        
    return out


In [45]:
results = {}

for task in tasks:
    task_results = {}
    task_metrics = metrics[task]
    for metric in task_metrics:
        
        # base metrics
        print(f"../../mt_dnn_exp_results/{exp}/{task}/base-{metric}.txt")
        f=open(f"../../mt_dnn_exp_results/{exp}/{task}/base-{metric}.txt", "r")
        lines = f.read().splitlines()
        task_results[f'base-{metric}'] = get_average_val(lines)
        
        # no layer metrics
        
        fine_tuning_metrics = []
        f=open(f"../../mt_dnn_exp_results/{exp}/{task}/no_layer-{metric}.txt", "r")

        lines = f.read().splitlines()
        fine_tuning_metrics.append(get_average_val(lines))
        
        # fine-tuned metrics
        
        log_file_prefix=''
        for i in reversed(range(int(num_layers/2), num_layers)):
            log_file_prefix += str(i)
            f=open(f"../../mt_dnn_exp_results/{exp}/{task}/{log_file_prefix}-{metric}.txt", "r")
            lines = f.read().splitlines()
            fine_tuning_metrics.append(get_average_val(lines))
            
            log_file_prefix +='_'
        
        task_results[f'{metric}'] = list(reversed(fine_tuning_metrics))
        
    results[task] = task_results

../../mt_dnn_exp_results/bert-base/CoLA/base-mcc.txt
../../mt_dnn_exp_results/bert-base/SST-2/base-acc.txt
../../mt_dnn_exp_results/bert-base/MRPC/base-f1.txt
../../mt_dnn_exp_results/bert-base/STS-B/base-spearmanr.txt
../../mt_dnn_exp_results/bert-base/QQP/base-f1.txt
../../mt_dnn_exp_results/bert-base/MNLI/base-acc.txt
../../mt_dnn_exp_results/bert-base/MNLI-MM/base-acc.txt
../../mt_dnn_exp_results/bert-base/QNLI/base-acc.txt
../../mt_dnn_exp_results/bert-base/RTE/base-acc.txt


In [46]:
x_axis = []

for i in range(int(num_layers/2), num_layers):
    x_axis.append(str(i))

x_axis.append("none")

In [47]:
def draw_graph(task, y_label, paper, base, reported):
    plt.figure(figsize=(10,6))
    plt.plot(x_axis, reported)
    
    plt.xlabel("layers")
    plt.ylabel(y_label)
    
    if paper == 0.0:    
        gap = max(reported) - min(reported)
        top = max(max(reported), base) + (gap*0.2)
        bottom = min(min(reported), base) - (gap*0.2)
    
        plt.ylim(bottom, top)

        plt.axhline(y=base, linestyle='--', c='green')
    else:
        gap = max(reported) - min(reported)
        top = max(max(reported), base, paper) + (gap*0.2)
        bottom = min(min(reported), base, paper) - (gap*0.2)
    
        plt.ylim(bottom, top)

        plt.axhline(y=base, linestyle='--', c='green')
        plt.axhline(y=paper, linestyle='--', c='red')
    
    plt.title(f'{exp}-{task} ({round(base,4)})')
    plt.savefig(f'images/{exp}/{task}', format='png', bbox_inches='tight')
    plt.show()

In [48]:
for task in tasks:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        reported = task_results[metric]
        base = task_results[f'base-{metric}']
        print_bold(task, metric, ': b -', round(base * 100, 2), 'h -',round(task_results[metric][0] * 100, 2), 'n -', round(task_results[metric][-1] * 100, 2))


CoLA mcc : b - 57.38 h - 55.69 n - 53.64


SST-2 acc : b - 92.78 h - 92.55 n - 91.86


MRPC f1 : b - 92.28 h - 92.28 n - 85.4


STS-B spearmanr : b - 90.88 h - 90.64 n - 89.54


QQP f1 : b - 88.38 h - 87.76 n - 87.18


MNLI acc : b - 84.44 h - 84.02 n - 76.65


MNLI-MM acc : b - 84.67 h - 84.46 n - 77.74


QNLI acc : b - 91.11 h - 90.94 n - 85.89


RTE acc : b - 78.41 h - 79.49 n - 71.05



In [51]:
import copy 

layer_90 = []
layer_95 = []

threshold_90 = 0.9
threshold_95 = 0.95
x_axis.reverse()

for task in tasks:
#     print_bold(task)
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        base = task_results[f'base-{metric}']
        reported = copy.deepcopy(task_results[metric])
        reported.reverse()
        
        flag_90 = True
        flag_95 = True

        for ind, val in enumerate(reported):

            if val/base > threshold_90 and flag_90:
                flag_90 = False
                layer_90.append(ind)
                results[task]['90%'] = ind

            if val/base > threshold_95 and flag_95:
                flag_95 = False
                layer_95.append(ind)
                results[task]['95%'] = ind

        if flag_90:
            print(task, "Fails to achieve 90% threshold", reported[-1]/base)
            layer_90.append(len(reported)-1)
            results[task]['90%'] = "-"

        if flag_95:
            print(task, "Fails to achieve 95% threshold", reported[-1]/base)
            layer_95.append(len(reported)-1)
            results[task]['95%'] = "-"


            
print(x_axis)
            
            
print(layer_90)
min_layer_ind_90 = max(layer_90)
print("layer_90 ", min_layer_ind_90, 'layer:', x_axis[min_layer_ind_90], round((1-(min_layer_ind_90/num_layers)) * 100, 2), '%')

print(layer_95)
min_layer_ind_95 = max(layer_95)
print("layer_95 ", min_layer_ind_95, 'layer:', x_axis[min_layer_ind_95], round((1-(min_layer_ind_95/num_layers)) * 100, 2), '%')


firsts = []
seconds = []
    
for task in tasks:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        base = task_results[f'base-{metric}']
        reported = copy.deepcopy(task_results[metric])
        reported.reverse()
        
        if task != "CoLA":
            first = round(100*reported[0]/base, 2)
            second = round(100*reported[1]/base, 2)
            firsts.append(first)
            seconds.append(second)
            
        print_bold(task, base)
        print('\t90', reported[min_layer_ind_90], round(reported[min_layer_ind_90]/base * 100, 2))
        print('\t95', reported[min_layer_ind_95], round(reported[min_layer_ind_95]/base * 100, 2))
        
print_bold(len(firsts), np.mean(firsts), np.mean(seconds), round(np.mean(seconds) - np.mean(firsts),2))

['6', '7', '8', '9', '10', '11', 'none']
[0, 0, 0, 0, 0, 0, 0, 0, 0]
layer_90  0 layer: 6 100.0 %
[5, 0, 1, 0, 0, 1, 1, 1, 1]
layer_95  5 layer: 11 58.33 %

CoLA 0.573754832442525

	90 0.5364114280465591 93.49
	95 0.562479932991527 98.03

SST-2 0.9277522935779816

	90 0.9185779816513762 99.01
	95 0.9259174311926606 99.8

MRPC 0.9227562669317466

	90 0.8539696446626991 92.55
	95 0.9184687817817112 99.54

STS-B 0.9087928989577281

	90 0.8954457153626455 98.53
	95 0.9054230987704198 99.63

QQP 0.8837812612880741

	90 0.8717620837936814 98.64
	95 0.8769679326611739 99.23

MNLI 0.84444218033622

	90 0.766479877738156 90.77
	95 0.8411003565970454 99.6

MNLI-MM 0.8466639544344995

	90 0.777379983726607 91.82
	95 0.8429617575264443 99.56

QNLI 0.9111111111111111

	90 0.8589419732747574 94.27
	95 0.9087314662273476 99.74

RTE 0.7841155234657039

	90 0.7104693140794224 90.61
	95 0.7913357400722022 100.92

8 94.525 98.2475 3.72



In [ ]:
for task in ["STS-B"]:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        
        print(task_results[metric][-1])
        print(task_results[metric][-2])

In [54]:
print("\\begin{table*}\n\\centering\n\\setlength{\\tabcolsep}{2pt}\n\\scalebox{0.88}{\n\\begin{tabular}{lc|ccccccc|cc}\n\\toprule[1pt]\ntask & base & 6-11 & 7-11 & 8-11 & 9-11 & 10-11 & 11-11 & none & 90\\% & 95\\%\\\\\n\\midrule")

for task in tasks:
    m = metrics[task][0]
    base_key = f"base-{m}"
    row = f"{task} & "
    
    row += "{:0.2f}".format(round(results[task][base_key] * 100, 2))
    
    for val in results[task][m]:
        row += " & {:0.2f}".format(round(val * 100,2))
        
    row += "& {}".format(results[task]["90%"])
    row += "& {}".format(results[task]["95%"])
        
    row += " \\\\"
    print(row)
    
print("\\end{tabular}\n}\n\\caption{MTDNN-BERT-base on GLUE}\n\\label{table:finetune-all}\n\\end{table*}")

\begin{table*}
\centering
\setlength{\tabcolsep}{2pt}
\scalebox{0.88}{
\begin{tabular}{lc|ccccccc|cc}
\toprule[1pt]
task & base & 6-11 & 7-11 & 8-11 & 9-11 & 10-11 & 11-11 & none & 90\% & 95\%\\
\midrule
CoLA & 57.38 & 55.69 & 56.25 & 54.28 & 54.11 & 54.22 & 54.22 & 53.64& 0& 5 \\
SST-2 & 92.78 & 92.55 & 92.59 & 92.34 & 92.27 & 92.22 & 92.04 & 91.86& 0& 0 \\
MRPC & 92.28 & 92.28 & 91.85 & 91.83 & 91.49 & 91.50 & 90.61 & 85.40& 0& 1 \\
STS-B & 90.88 & 90.64 & 90.54 & 90.47 & 90.30 & 90.15 & 89.97 & 89.54& 0& 0 \\
QQP & 88.38 & 87.76 & 87.70 & 87.63 & 87.54 & 87.50 & 87.40 & 87.18& 0& 0 \\
MNLI & 84.44 & 84.02 & 84.11 & 84.19 & 83.91 & 83.72 & 83.05 & 76.65& 0& 1 \\
MNLI-MM & 84.67 & 84.46 & 84.30 & 84.25 & 84.16 & 83.90 & 83.34 & 77.74& 0& 1 \\
QNLI & 91.11 & 90.94 & 90.87 & 90.66 & 90.39 & 90.18 & 89.25 & 85.89& 0& 1 \\
RTE & 78.41 & 79.49 & 79.13 & 78.48 & 76.82 & 75.60 & 75.23 & 71.05& 0& 1 \\
\end{tabular}
}
\caption{MTDNN-BERT-base on GLUE}
\label{table:finetune-all}
\end{table*}
